In [4]:
import yaml
import platform
import pandas as pd
import glob
from PIL import Image
import brambox as bb
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
def check_os():
    os = platform.system()

    if os == 'Darwin':
        return "MacOS"
    elif os == 'Linux':
        return "Linux"
    else:
        return "Unknown OS"
    
operating_system = check_os()


if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [17]:
from brambox.io.parser.detection import CocoParser

# Load detections

det = bb.io.load(CocoParser, '/Users/johnny/Projects/small-fast-detector/runs/detect/val/predictions.json')
print('detections:')
det['image'] = det['image'].astype(str).str.lstrip('0').astype(int)
display(det.head())
from brambox.io.parser.annotation import CocoParser
# Load annotations
anno = bb.io.load(CocoParser(add_image_dims=True), '/Users/johnny/Projects/small-fast-detector/inference_tools/Evaluation/datasets/Client_Validation_Set/annotations/instances_val2017.json')
anno['image'] = anno['image'].astype(str).str.lstrip('0').astype(int)
print('annotations:')
display(anno.head())

# save dataframes
det.to_csv('/Users/johnny/Projects/small-fast-detector/runs/detect/val/detections.csv', index=False)

anno.to_csv('/Users/johnny/Projects/small-fast-detector/inference_tools/Evaluation/datasets/Client_Validation_Set/annotations/annotations.csv', index=False)

detections:


,image,class_label,id,x_top_left,y_top_left,width,height,confidence
0,29,3,NaN,353.822,75.017,1267.850,975.556,0.93743
1,29,0,NaN,414.993,232.212,45.999,49.189,0.00237
2,406,3,NaN,570.447,144.623,217.384,101.411,0.84726
3,406,4,NaN,570.447,144.623,217.384,101.411,0.00186
4,406,3,NaN,975.589,202.266,34.866,21.118,0.00332


annotations:


,image,class_label,id,x_top_left,y_top_left,width,height,occluded,truncated,lost,difficult,ignore,image_width,image_height
0,00029,uav,5480.0,359.0,67.0,1259.0,992.0,0.0,0.0,False,False,False,1920,1080
1,00406,uav,4919.0,572.0,145.0,210.0,98.0,0.0,0.0,False,False,False,1920,1080
2,00407,uav,5156.0,1072.0,249.0,137.0,64.0,0.0,0.0,False,False,False,1920,1080
3,00408,uav,2327.0,1166.0,329.0,152.0,61.0,0.0,0.0,False,False,False,1920,1080
4,00409,uav,2027.0,936.0,673.0,116.0,44.0,0.0,0.0,False,False,False,1920,1080


In [59]:
def calculate_area(row):
    return row['width'] * row['height']

def iou(box_a, box_b):
    xA = max(box_a[0], box_b[0])
    yA = max(box_a[1], box_b[1])
    xB = min(box_a[2], box_b[2])
    yB = min(box_a[3], box_b[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = (box_a[2] - box_a[0]) * (box_a[3] - box_a[1])
    boxBArea = (box_b[2] - box_b[0]) * (box_b[3] - box_b[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def calculate_map(detected, actual, class_labels):
    aps = []
    if detected.empty or 'class_label' not in detected.columns:
        return 0

    for label in class_labels:
        if label not in detected['class_label'].values:
            aps.append(0)
            continue

        dc = detected[detected.class_label == label]
        ac = actual[actual.class_label == label]

        ap_coco = []
        for iou_threshold in range(50, 100, 5):
            if dc.empty:
                ap_coco.append(0)
                continue

            pr = bb.stat.pr(dc, ac, iou_threshold / 100, smooth=True)
            ap_coco.append(bb.stat.auc_interpolated(pr))

        aps.append(sum(ap_coco) / len(ap_coco))

    mAP_coco = sum(aps) / len(aps) if aps else 0
    return mAP_coco



def calculate_metrics(detected, actual, catIds, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = len(actual)

    for det in detected.itertuples():
        max_iou = 0
        for gt in actual.itertuples():
            current_iou = iou([det.x_top_left, det.y_top_left, det.x_top_left + det.width, det.y_top_left + det.height], 
                              [gt.x_top_left, gt.y_top_left, gt.x_top_left + gt.width, gt.y_top_left + gt.height])
            max_iou = max(max_iou, current_iou)

        if max_iou >= iou_threshold:
            true_positives += 1
            false_negatives -= 1
        else:
            false_positives += 1

    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    mAP = calculate_map(detected, actual, catIds)

    return recall, precision, mAP

image_stats = pd.DataFrame(columns=['name', 'width', 'height', 'num_of_gt_objects', 'lowest_area', 'biggest_area', 'num_of_predicted_objects', 'recall', 'mAP'])

det_grouped = det.groupby('image', observed=True)
anno_grouped = anno.groupby('image', observed=True)

class_labels = anno['class_label'].unique().tolist()

total_images = set(anno['image'].unique().tolist() + det['image'].unique().tolist())
for image_id in total_images:
    if image_id in anno_grouped.groups:
        image_data = anno_grouped.get_group(image_id).copy()
        width = image_data.iloc[0]['image_width']
        height = image_data.iloc[0]['image_height']
    else:
        continue

    num_of_gt_objects = len(image_data)
    image_data['area'] = image_data.apply(calculate_area, axis=1)
    lowest_area = image_data['area'].min() if not image_data['area'].empty else np.nan
    biggest_area = image_data['area'].max() if not image_data['area'].empty else np.nan

    if image_id in det_grouped.groups:
        det_data = det_grouped.get_group(image_id)
        num_of_predicted_objects = len(det_data)
    else:
        det_data = pd.DataFrame()
        num_of_predicted_objects = 0

    recall, precision, mAP = calculate_metrics(det_data, image_data, class_labels)

    if not det_data.empty or num_of_gt_objects > 0:
        new_row = pd.DataFrame([{
            'name': image_id,
            'width': width,
            'height': height,
            'num_of_gt_objects': num_of_gt_objects,
            'lowest_area': lowest_area,
            'biggest_area': biggest_area,
            'num_of_predicted_objects': num_of_predicted_objects,
            'recall': recall,
            'mAP': mAP
        }])
        image_stats = pd.concat([image_stats, new_row], ignore_index=True)

/var/folders/d_/mv_tvljd4wz2dmh2c7kcl6xm0000gn/T/ipykernel_19386/669981857.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  image_stats = pd.concat([image_stats, new_row], ignore_index=True)


In [57]:
image_stats.sort_values(by=['name'], inplace=True)
image_stats

,name,width,height,num_of_gt_objects,lowest_area,biggest_area,num_of_predicted_objects,recall,mAP
127,1000,1920,1080,1,9225.0,9225.0,152,0.000000,0.0
488,1001,1920,1080,14,924.0,7261.0,300,3.214286,0.0
696,1002,1920,1080,14,736.0,6111.0,300,2.857143,0.0
1717,1003,1920,1080,9,288.0,8512.0,274,2.888889,0.0
37,1004,1920,1080,11,704.0,5069.0,269,1.727273,0.0
...,...,...,...,...,...,...,...,...,...
1871,995,1920,1080,7,504.0,3185.0,266,2.285714,0.0
1981,996,1920,1080,8,615.0,3204.0,258,2.500000,0.0
1000,997,1920,1080,1,3784.0,3784.0,159,3.000000,0.0
1084,998,1920,1080,1,7920.0,7920.0,188,3.000000,0.0


In [ ]:
from ultralytics import YOLO
import os
import glob
import pandas as pd
from PIL import Image

# Suponiendo que 'root_path' está definido
images_directory = root_path + 'datasets/custom_dataset_v2/images/val/'
labels_directory = root_path + 'datasets/custom_dataset_v2/labels/val/'

image_files = glob.glob(images_directory + '*.jpg')
model = YOLO('../inference_tools/Evaluation/models/detector_best.pt', task='detect')

df_rows = []

def get_image_resolution(image_path):
    with Image.open(image_path) as img:
        return img.size

for image_file in image_files:
    image_resolution = get_image_resolution(image_file)
    base_name = os.path.basename(image_file).replace('.jpg', '')
    label_file = os.path.join(labels_directory, base_name + '.txt')
    
    if os.path.exists(label_file):
        with open(label_file, 'r') as file:
            annotation_data = file.readlines()
        
        for line in annotation_data:
            class_id, x_center, y_center, width, height = line.strip().split()
            object_width = int(float(width) * image_resolution[0])
            object_height = int(float(height) * image_resolution[1])

            df_rows.append({
                'file_name': base_name + '.jpg',
                'class_id': int(class_id),
                'x_center': float(x_center),
                'y_center': float(y_center),
                'width': float(width),
                'height': float(height),
                'res_width': image_resolution[0],
                'res_height': image_resolution[1],
                'obj_width': object_width,
                'obj_height': object_height,
                'image_path': image_file,
                'label_path': label_file,
            })

df_annotations = pd.DataFrame(df_rows)

def calculate_area(width, height):
    return width * height

def predict_yolov8(image_path, label_path=None):
    results = model(image_path, size=640)
    
    predictions = []  
    metrics = {'recall': 0.0, 'map': 0.0}  
    return predictions, metrics

# Procesamiento adicional para obtener las métricas y predicciones
for index, row in df_annotations.iterrows():
    predictions, metrics = predict_yolov8(row.image_path)

    df_annotations.at[index, 'num_of_predicted_objects'] = len(predictions)
    df_annotations.at[index, 'recall'] = metrics['recall']
    df_annotations.at[index, 'map'] = metrics['map']

    areas = [calculate_area(obj.obj_width, obj.obj_height) for obj in df_annotations.itertuples() if obj.image_path == row.image_path]
    if areas:
        df_annotations.at[index, 'lowest_area'] = min(areas)
        df_annotations.at[index, 'biggest_area'] = max(areas)

df_annotations['num_of_gt_objects'] = df_annotations.groupby('file_name')['file_name'].transform('count')

print(df_annotations.head())

In [ ]:
df_annotations.to_csv('/data-fast/108-data3/ierregue/datasets/custom_dataset_v1/annotations_valid.csv', index=False)

In [ ]:
full_hd_new_data = df_annotations[(df_annotations['res_width'] == 1920) & (df_annotations['res_height'] == 1080)].copy()